Import neccesary packages like pandas, numpy, and matplotlib.pyplot

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Since each dataset is monthly, write some code to combine all 12 data sets for 2019.

In [ ]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
different_csvs = []
for month in months:
  monthly = pd.read_csv(f'https://raw.githubusercontent.com/i-am-darshil/Data-Analysis-With-Pandas/main/SalesData/Sales_{month}_2019.csv')
  different_csvs.append(monthly)

df = pd.concat(different_csvs, ignore_index=True)

Determine whether there are null values in the combined dataset.

In [ ]:
nan_count = df.isna().sum(axis=0)
print(nan_count)
print(df.info())

Clean dataset by:
1. dropping null values
2. reseting the index so there are no missing numbers

In [ ]:
df_clean = df.dropna()
df_reset = df_clean.reset_index(drop=True)
df_reset.info()

Remove the values that aren't null but aren't inputed as an actual value.

In [ ]:
df_reset.drop(df_reset[df_reset['Quantity Ordered'] == "Quantity Ordered"].index, inplace=True)

Change the column types to more appropriate types. So for example if the column holds numeric values, make sure the type is a float/integer instead of a string/object.

In [ ]:
df_reset['Quantity Ordered'] = df_reset['Quantity Ordered'].astype(int)
df_reset['Price Each'] = df_reset['Price Each'].astype(float)
df_reset.info()


In [ ]:
print(df_reset)

Create a column for the total transaction amount per order. Get this by identifying the price of each product and multiplying it by the quantity ordered.

In [ ]:
df_reset["Total Price"] = df_reset["Quantity Ordered"] * df_reset["Price Each"]
df_reset

Create a column for the state the order was placed in so you can do state-wide analysis.

In [ ]:
df_reset["State"] = df_reset['Purchase Address'].str[-8:-6]
df_reset

Use Matplotlib.pyplot to create a graph that identifies which state generates the most revenue.

In [ ]:
state_totals = df_reset.groupby('State')['Total Price'].sum() / 1000
state_totals = state_totals.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
graph = state_totals.plot(kind='bar', color='skyblue')

plt.title('Total Transaction Amount by State')
plt.xlabel('State')
plt.ylabel('Total Transaction Amount (K)')

graph.ticklabel_format(style='plain', axis='y')

plt.show()

Figure out which product generates the most revenue for ht ebusiness by creating a graph that groups the price by product and putting that on a graph. Also figure out which product is being ordered the most. (Same thing but with quantity ordered)

In [ ]:
products_count = df_reset.groupby('Product')['Total Price'].sum()/1000
products_count = products_count.sort_values(ascending=False)
products_count

plt.figure(figsize=(14, 10))
graph2 = products_count.plot(kind='bar', color='magenta')

plt.title('Total Transaction Amount by Product (Thousand Dollars)')
plt.xlabel('Product')
plt.ylabel('Total Transaction Amount')

graph2.ticklabel_format(style='plain', axis='y')

plt.show()

quantity_count = df_reset.groupby('Product')['Quantity Ordered'].sum()
quantity_count = products_count.sort_values(ascending=False)
quantity_count

plt.figure(figsize=(14, 10))
graph2 = quantity_count.plot(kind='bar', color='teal')

plt.title('Quantity Ordered by Product')
plt.xlabel('Product')
plt.ylabel('Total Quantity Amount by Product')

graph2.ticklabel_format(style='plain', axis='y')

plt.show()

Create graph of all the revenue generated in April so you can determine when to advertise.

In [ ]:

df_reset['Order Date'] = pd.to_datetime(df_reset['Order Date'], format='%m/%d/%y %H:%M')
df_reset['Day'] = df_reset['Order Date'].dt.day

daily_sales = df_reset.groupby('Day')['Total Price'].sum()/1000
min_day = daily_sales.idxmin()
print("You should aim to advertise on the", min_day, "of April, since sales are the lowest that day.")

plt.figure(figsize=(10, 6))
daily_sales.plot(kind='bar')
plt.xlabel('Day in April 2019')
plt.ylabel('Total Price (Thousand Dollars)')
plt.title('Total Sales by Day in April 2019')
plt.xticks(rotation=0)
plt.show()

In [ ]:
df.head(10)

Make a graph to understand what time of day you make the most money.

In [ ]:
df_reset['hour']= pd.to_datetime(df_reset['Order Date']).dt.hour
df_reset.head(10)
sale_by_hour = df_reset.groupby('hour')['Total Price'].sum()
plt.figure(figsize=(14, 10))
graph3 = sale_by_hour.plot(kind='bar', color='maroon')

plt.title('Total Transaction Amount by hour of day')
plt.xlabel('Hour')
plt.ylabel('Total Transaction Amount')

graph3.ticklabel_format(style='plain', axis='y')

plt.show()


In [ ]:
df_reset['hour']= pd.to_datetime(df_reset['Order Date']).dt.hour
df_reset.head(10)
sale_by_hour = df_reset.groupby('hour')['Quantity Ordered'].sum()
plt.figure(figsize=(14, 10))
graph4 = sale_by_hour.plot(kind='bar', color='maroon')

plt.title('Total Transaction Amount by hour of day')
plt.xlabel('Hour')
plt.ylabel('Total Transaction Amount')

graph4.ticklabel_format(style='plain', axis='y')

plt.show()


Figure out which order ID's have duplicates, these orders are the ones where customers bought multiple things at once.

In [ ]:
duplicates = df_reset['Order ID'][df['Order ID'].duplicated()]

# Display the duplicate values and their counts
duplicate_counts = duplicates.value_counts()
duplicate_counts

Use the duplicated order ID to figure out which products are commonly sold together.

In [ ]:
# find which products are frequently sold together using the duplicates in orderID

df_duplicated = df_reset[df_reset.duplicated(subset=['Order ID'], keep=False)]
grouped_orders = df_duplicated.groupby('Order ID')['Product'].apply(list)
print(grouped_orders.head())
product_combinations = {}

for order in grouped_orders:
  # Sort the products to avoid counting (A, B) and (B, A) separately
  order = sorted(order)
  # Convert the list to a tuple to make it hashable
  order_tuple = tuple(order) # Convert list to tuple
  if order_tuple in product_combinations: # Check if tuple is in dictionary
      product_combinations[order_tuple] += 1
  else:
      product_combinations[order_tuple] = 1

sorted_combinations = sorted(product_combinations.items(), key=lambda x: x[1], reverse=True)

for order, count in sorted_combinations[:10]:  # Display top 10 combinations
  print(order, count)